# Hadoop Installation and Testing

> **WARNING: Be careful, the execution of this notebook can compromise your virtual machines. Do not execute any cell twice: please start from the very first cell if you have problems.**

Load commands from `commands.py`.

In [46]:
%reload_ext autoreload
%autoreload 1
%aimport commands

Functions we will use

In [47]:
def run_ssh(host, *command):
    """
    :param host: ip address
    :type f: string
    :param command: command to execute
    :type command: string

    Execute with SSH the commands on host as the 'hadoop' user, displaying information
    """
    print('===== \x1b[31m' + 'Started on ' + host + '\x1b[0m =====')
    for cmd in command:
        print(cmd)
        !ssh hadoop@{host} {cmd}
    print('===== \x1b[31m' + 'Completed on ' + host + '\x1b[0m =====')

Generally Hadoop can be run in three modes.

1. **Standalone (or local) mode**: There are no daemons used in this mode. Hadoop uses the local file system as a substitute for HDFS file system. The jobs will run as if there is 1 mapper and 1 reducer.
2. **Pseudo-distributed mode**: All the daemons run on a single machine and this setting mimics the behavior of a cluster. All the daemons run on your machine locally using the HDFS protocol. There can be multiple mappers and reducers.
3. **Fully-distributed mode**: This is how Hadoop runs on a real cluster.

In these notes we will describe how to set up an [Hadoop 3](https://hadoop.apache.org) installation to work with.
We will set up a *fully-distributed cluster* on your assigned virtual machines.

The core of Hadoop is composed by two main subsystem:

* the **Hadoop Distributed File System** (HDFS), responsible for the distributed data management
* the **Yet Another Resource Negotiator** (YARN), responsible for the distributed code execution

Both subsystems are implemented according to the **master-workers** architecture.

![Master-workers Architecture](https://raw.githubusercontent.com/mesham/epython/master/docs/masterworker.png)

Both HDFS and YARN have their own terminology for master and worker nodes.

|    | Master         | Worker      |
|:--:|:--------------:|:-----------:|
|HDFS| NameNode       | DataNode    |
|YARN| ResouceManager | NodeManager |

While the masters of HDFS and YARN can, in principle, be located on different machines, we will install the HDFS and YARN masters on a single machine, and install the HDFS and YARN workers on all machines (*including the machine hosting the masters).

This notebook contains the steps necessary to set up and configure correctly Hadoop on our virtual machines.
In particular:

1. We will [download & install](#download) Hadoop on all our virtual machines.
2. We will [configure](#namenode) a virtual machine to host the HDFS and YARN masters and workers.
3. We will [configure](#datanode) the remaining virtual machines to host the HDFS and YARN workers.
4. We will [test](#test) your newly install Hadoop cluster.

## 0. Prerequisites <a name="prereq"/>

**a.** Populate the following dictionary with the IP addresses (as keys) and the hostnames (as values) of all your virtual machines.

In [48]:
VMS = {'172.16.3.79': 'datanode1',
       '172.16.3.80': 'datanode2', 
       '172.16.3.81': 'namenode'}

**b.** Populate the following variable with the IP address of the virtual machine with the namenode role.

In [49]:
NAMENODE_IP = '172.16.3.81'

**c.** Populate the following variable with the IP address of the remaining virtual machines.

In [50]:
REMAINING_IPS = [
    '172.16.3.79',
    '172.16.3.80'
]

## 1. Download and install Hadoop <a name="download"/>
---
**a.** Download [hadoop-3.1.3.tar.gz](https://archive.apache.org/dist/hadoop/common/hadoop-3.1.3/hadoop-3.1.3.tar.gz) in your home folder on your virtual machines.

In [7]:
for host in VMS:
    run_ssh(host, commands.WGET_CMD)

===== Started on 172.16.3.79 =====
wget --progress=bar:force -c -O /home/hadoop/hadoop.tar.gz https://archive.apache.org/dist/hadoop/common/hadoop-3.1.3/hadoop-3.1.3.tar.gz
--2021-04-13 17:31:36--  https://archive.apache.org/dist/hadoop/common/hadoop-3.1.3/hadoop-3.1.3.tar.gz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 338075860 (322M) [application/x-gzip]
Saving to: ‘/home/hadoop/hadoop.tar.gz’

/home/hadoop/hadoop 100%[===================>] 322.41M  13.4MB/s    in 14s     

2021-04-13 17:31:49 (23.6 MB/s) - ‘/home/hadoop/hadoop.tar.gz’ saved [338075860/338075860]

===== Completed on 172.16.3.79 =====
===== Started on 172.16.3.80 =====
wget --progress=bar:force -c -O /home/hadoop/hadoop.tar.gz https://archive.apache.org/dist/hadoop/common/hadoop-3.1.3/hadoop-3.1.3.tar.gz
--2021-04-13 17:32:05--  h

**b.** Decompress the Hadoop package you can use the following command:

In [8]:
for host in VMS:
    run_ssh(host, commands.TAR_CMD, commands.RM_CMD)

===== Started on 172.16.3.79 =====
tar -xvf hadoop.tar.gz --directory=/opt/hadoop --exclude=hadoop-3.1.0/share/doc --strip 1 > /dev/null
rm /home/hadoop/hadoop.tar.gz
===== Completed on 172.16.3.79 =====
===== Started on 172.16.3.80 =====
tar -xvf hadoop.tar.gz --directory=/opt/hadoop --exclude=hadoop-3.1.0/share/doc --strip 1 > /dev/null
rm /home/hadoop/hadoop.tar.gz
===== Completed on 172.16.3.80 =====
===== Started on 172.16.3.81 =====
tar -xvf hadoop.tar.gz --directory=/opt/hadoop --exclude=hadoop-3.1.0/share/doc --strip 1 > /dev/null
rm /home/hadoop/hadoop.tar.gz
===== Completed on 172.16.3.81 =====


**c.** There are environment settings that will be used by Hadoop. The following commands append the correct environment variables to your `/home/hadoop/.bashrc` files:

In [24]:
for host in VMS:
    !ssh hadoop@{host} "sed -i '1,10 s/^/#/' ~/.bashrc"
    !ssh hadoop@{host} 'printf "%s\n" {commands.get_bashrc()} >> ~/.bashrc'

cazzo
cazzo


**d.** The following commands check Hadoop installation (you should see no errors):

In [25]:
for host in VMS:
    run_ssh(host, 'hadoop version')

===== Started on 172.16.3.79 =====
hadoop version
Hadoop 3.1.3
Source code repository https://gitbox.apache.org/repos/asf/hadoop.git -r ba631c436b806728f8ec2f54ab1e289526c90579
Compiled by ztang on 2019-09-12T02:47Z
Compiled with protoc 2.5.0
From source with checksum ec785077c385118ac91aadde5ec9799
This command was run using /opt/hadoop/share/hadoop/common/hadoop-common-3.1.3.jar
===== Completed on 172.16.3.79 =====
===== Started on 172.16.3.80 =====
hadoop version
cazzo
Hadoop 3.1.3
Source code repository https://gitbox.apache.org/repos/asf/hadoop.git -r ba631c436b806728f8ec2f54ab1e289526c90579
Compiled by ztang on 2019-09-12T02:47Z
Compiled with protoc 2.5.0
From source with checksum ec785077c385118ac91aadde5ec9799
This command was run using /opt/hadoop/share/hadoop/common/hadoop-common-3.1.3.jar
===== Completed on 172.16.3.80 =====
===== Started on 172.16.3.81 =====
hadoop version
Hadoop 3.1.3
Source code repository https://gitbox.apache.org/repos/asf/hadoop.git -r ba631c436b806728

## 2. Configure the `hadoop-namenode` machine <a name="namenode"/>

**a.** Update the `core-site.xml` file located at `/opt/hadoop/etc/hadoop/` to define the name node URI on this machine.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>fs.defaultFS</name>
    <value>hdfs://hadoop-namenode:9820/</value>
  </property>
</configuration>
```
The following command updates the file automatically.

In [26]:
!ssh hadoop@{NAMENODE_IP} 'printf "%s\n" {commands.get_namenode_core_site(VMS[NAMENODE_IP])} > /opt/hadoop/etc/hadoop/core-site.xml'

**b.** Update the `hdfs-site.xml` file located at `/opt/hadoop/etc/hadoop/` to define the path on the local filesystem where the name node stores the namespace and transactions logs persistently and to configure the HDFS subsystem.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>dfs.namenode.name.dir</name>
    <value>file:///opt/hdfs/namenode</value>
  </property>
  <property>
    <name>dfs.datanode.data.dir</name>
    <value>file:///opt/hdfs/datanode</value>
  </property>
  <property>
    <name>dfs.replication</name>
    <value>2</value>
  </property>
  <property>
    <name>dfs.permissions</name>
    <value>false</value>
  </property>
  <property>
    <name>dfs.datanode.use.datanode.hostname</name>
    <value>false</value>
  </property>
</configuration>
```
The following command updates the file automatically.

In [44]:
!ssh hadoop@{NAMENODE_IP} 'printf "%s\n" {commands.get_namenode_hdfs_site()} > /opt/hadoop/etc/hadoop/hdfs-site.xml'

**c.** Update the `yarn-site.xml` file located at `/opt/hadoop/etc/hadoop` to configure the YARN subsystem.
The file must contain the following lines.

```xml
<configuration>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.resourcemanager.hostname</name>
    <value>{hadoop_namenode}</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services.mapreduce.shuffle.class</name>
    <value>org.apache.hadoop.mapred.ShuffleHandler</value>
  </property>
  <property>
    <name>yarn.nodemanager.local-dirs</name>
    <value>file:///opt/yarn/local</value>
  </property>
  <property>
    <name>yarn.nodemanager.log-dirs</name>
    <value>file:///opt/yarn/logs</value>
  </property>
  <property>
    <name>yarn.nodemanager.resource.memory-mb</name>
    <value>1536</value>
  </property>
  <property>
    <name>yarn.scheduler.maximum-allocation-mb</name>
    <value>1536</value>
  </property>
  <property>
    <name>yarn.scheduler.minimum-allocation-mb</name>
    <value>128</value>
  </property>
  <property>
    <name>yarn.nodemanager.vmem-check-enabled</name>
    <value>false</value>
  </property>
</configuration>
```

The following command updates the file automatically.

In [51]:
!ssh hadoop@{NAMENODE_IP} 'printf "%s\n" {commands.get_namenode_yarn_site(VMS[NAMENODE_IP])} > /opt/hadoop/etc/hadoop/yarn-site.xml'

**d.** Update the `mapred-site.xml` file located at `/opt/hadoop/etc/hadoop` to configure the MAPREDUCE subsystem.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.jobhistory.address</name>
    <value>{namenode_hostname}:10020</value>
  </property>
  <property>
    <name>mapreduce.jobhistory.webapp.address</name>
    <value>{namenode_hostname}:19888</value>
  </property>
  <property>
    <name>mapreduce.jobhistory.intermediate-done-dir</name>
    <value>/mr-history/tmp</value>
  </property>
  <property>
    <name>mapreduce.jobhistory.done-dir</name>
    <value>/mr-history/done</value>
  </property>
  <property>
    <name>yarn.app.mapreduce.am.env</name>
    <value>HADOOP_MAPRED_HOME=/opt/hadoop</value>
  </property>
  <property>
    <name>mapreduce.map.env</name>
    <value>HADOOP_MAPRED_HOME=/opt/hadoop</value>
  </property>
  <property>
    <name>mapreduce.reduce.env</name>
    <value>HADOOP_MAPRED_HOME=/opt/hadoop</value>
  </property>
  <property>
    <name>yarn.app.mapreduce.am.resource.mb</name>
    <value>512</value>
  </property>
  <property>
    <name>mapreduce.map.memory.mb</name>
    <value>256</value>
  </property>
  <property>
    <name>mapreduce.reduce.memory.mb</name>
    <value>256</value>
  </property>
</configuration>
```

The following command updates the file automatically.

In [29]:
!ssh hadoop@{NAMENODE_IP} 'printf "%s\n" {commands.get_namenode_mapred_site(VMS[NAMENODE_IP])} > /opt/hadoop/etc/hadoop/mapred-site.xml'

> If your machines have more than 2 GB of RAM or if you are interested in the numbers we specified in the YARN and MAPRED configuration files, check the Append A on the [Hadoop Memory Allocaltion](#memory).

**e.** Update the `workers` file located in `/opt/hadoop/etc/hadoop` to define the MAPREDUCE workers.
With our virtual machines listed [here](#prereq), the file must contain the following lines.
```
172.16.0.225
172.16.0.221
172.16.0.224
```
The following command updates the file automatically.

In [30]:
!ssh hadoop@{NAMENODE_IP} 'printf "%s\n" {commands.get_workers(VMS)} > /opt/hadoop/etc/hadoop/workers'

## 3. Configure the `hadoop-datanode` machines <a name="datanode"/>

**a.** Update the `core-site.xml` file located at `/opt/hadoop/etc/hadoop/` to define the name node URI on thie other datanodes.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>fs.defaultFS</name>
    <value>hdfs://hadoop-namenode:9820/</value>
  </property>
</configuration>
```
The following command updates the file automatically.

In [31]:
for host in REMAINING_IPS:
    !ssh hadoop@{host} 'printf "%s\n" {commands.get_datanode_core_site(VMS[NAMENODE_IP])} > /opt/hadoop/etc/hadoop/core-site.xml'

**b.** Update the `hdfs-site.xml` file located at `/opt/hadoop/etc/hadoop/` to configure the HDFS subsystem.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>dfs.datanode.data.dir</name>
    <value>file:///opt/hdfs/datanode</value>
  </property>
  <property>
    <name>dfs.replication</name>
    <value>2</value>
  </property>
  <property>
    <name>dfs.permissions</name>
    <value>false</value>
  </property>
  <property>
    <name>dfs.datanode.use.datanode.hostname</name>
    <value>false</value>
  </property>
</configuration>
```
The following command updates the file automatically.

In [32]:
for host in REMAINING_IPS:
    !ssh hadoop@{host} 'printf "%s\n" {commands.get_datanode_hdfs_site()} > /opt/hadoop/etc/hadoop/hdfs-site.xml'

**c.** Update the `yarn-site.xml` file located at `/opt/hadoop/etc/hadoop` to configure the YARN subsystem.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.resourcemanager.hostname</name>
    <value>{namenode_hostname}</value>
  </property>
  <property>
    <name>yarn.nodemanager.resource.memory-mb</name>
    <value>1536</value>
  </property>
  <property>
    <name>yarn.scheduler.maximum-allocation-mb</name>
    <value>1536</value>
  </property>
  <property>
    <name>yarn.scheduler.minimum-allocation-mb</name>
    <value>128</value>
  </property>
  <property>
    <name>yarn.nodemanager.vmem-check-enabled</name>
    <value>false</value>
  </property>
</configuration>
```
The following command updates the file automatically.

In [52]:
for host in REMAINING_IPS:
    !ssh hadoop@{host} 'printf "%s\n" {commands.get_datanode_yarn_site(VMS[NAMENODE_IP])} > /opt/hadoop/etc/hadoop/yarn-site.xml'

**d.** Update the `mapred-site.xml` file located at `/opt/hadoop/etc/hadoop` to configure the MAPREDUCE subsystem.
The file must contain the following lines.
```xml
<configuration>
  <property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>yarn.app.mapreduce.am.env</name>
    <value>HADOOP_MAPRED_HOME=/opt/hadoop</value>
  </property>
  <property>
    <name>mapreduce.map.env</name>
    <value>HADOOP_MAPRED_HOME=/opt/hadoop</value>
  </property>
  <property>
    <name>mapreduce.reduce.env</name>
    <value>HADOOP_MAPRED_HOME=/opt/hadoop</value>
  </property>
  <property>
    <name>yarn.app.mapreduce.am.resource.mb</name>
    <value>512</value>
  </property>
  <property>
    <name>mapreduce.map.memory.mb</name>
    <value>256</value>
  </property>
  <property>
    <name>mapreduce.reduce.memory.mb</name>
    <value>256</value>
  </property>
</configuration>
```

The following command updates the file automatically.

In [53]:
for host in REMAINING_IPS:
    !ssh hadoop@{host} 'printf "%s\n" {commands.get_datanode_mapred_site()} > /opt/hadoop/etc/hadoop/mapred-site.xml'

> If your machines have more than 2 GB of RAM or if you are interested in the numbers we specified in the YARN and MAPRED configuration files, check the Append A on the [Hadoop Memory Allocaltion](#memory).

As a summary, please double check the content of the files list in the following picture, on the cluster machines:

![Hadoop Configuration Files](hadoop_conf.jpg)

## 4. Start, test and stop Hadoop <a name="test"/>

**From now, all commands will be issued from the `hadoop-namenode` virtual machine.**

**a**. Delete the contents of the local HDFS file system.
Note: **This causes the loss of all information stored in HDFS**.

In [38]:
for host in VMS:
    run_ssh(host, 'rm -rf /opt/hdfs/namenode/*')
    run_ssh(host, 'rm -rf /opt/hdfs/datanode/*')
    
host = NAMENODE_IP
!ssh hadoop@{host} 'stop-dfs.sh'

===== Started on 172.16.3.79 =====
rm -rf /opt/hdfs/namenode/*
zsh:1: no matches found: /opt/hdfs/namenode/*
===== Completed on 172.16.3.79 =====
===== Started on 172.16.3.79 =====
rm -rf /opt/hdfs/datanode/*
zsh:1: no matches found: /opt/hdfs/datanode/*
===== Completed on 172.16.3.79 =====
===== Started on 172.16.3.80 =====
rm -rf /opt/hdfs/namenode/*
zsh:1: no matches found: /opt/hdfs/namenode/*
===== Completed on 172.16.3.80 =====
===== Started on 172.16.3.80 =====
rm -rf /opt/hdfs/datanode/*
zsh:1: no matches found: /opt/hdfs/datanode/*
===== Completed on 172.16.3.80 =====
===== Started on 172.16.3.81 =====
rm -rf /opt/hdfs/namenode/*
zsh:1: no matches found: /opt/hdfs/namenode/*
===== Completed on 172.16.3.81 =====
===== Started on 172.16.3.81 =====
rm -rf /opt/hdfs/datanode/*
zsh:1: no matches found: /opt/hdfs/datanode/*
===== Completed on 172.16.3.81 =====
Stopping namenodes on [namenode]
Stopping datanodes
Stopping secondary namenodes [namenode]


**b**. Format the HDFS filesystem at the namenode.

In [39]:
host = NAMENODE_IP
!ssh hadoop@{host} 'hdfs namenode -format -force'

2021-04-13 18:08:00,651 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = namenode/172.16.3.81
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.1.3
STARTUP_MSG:   classpath = /opt/hadoop/etc/hadoop:/opt/hadoop/share/hadoop/common/lib/jersey-json-1.19.jar:/opt/hadoop/share/hadoop/common/lib/jackson-jaxrs-1.9.13.jar:/opt/hadoop/share/hadoop/common/lib/jackson-annotations-2.7.8.jar:/opt/hadoop/share/hadoop/common/lib/netty-3.10.5.Final.jar:/opt/hadoop/share/hadoop/common/lib/commons-logging-1.1.3.jar:/opt/hadoop/share/hadoop/common/lib/jetty-security-9.3.24.v20180605.jar:/opt/hadoop/share/hadoop/common/lib/re2j-1.1.jar:/opt/hadoop/share/hadoop/common/lib/error_prone_annotations-2.2.0.jar:/opt/hadoop/share/hadoop/common/lib/jackson-databind-2.7.8.jar:/opt/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar:/opt/hadoop/share/hadoop/common/lib/jetty-util-9.3.24.v201806

2021-04-13 18:08:00,761 INFO namenode.NameNode: createNameNode [-format, -force]
Formatting using clusterid: CID-dcfa04bc-2017-40b8-9a87-522b499cdedc
2021-04-13 18:08:01,235 INFO namenode.FSEditLog: Edit logging is async:true
2021-04-13 18:08:01,266 INFO namenode.FSNamesystem: KeyProvider: null
2021-04-13 18:08:01,268 INFO namenode.FSNamesystem: fsLock is fair: true
2021-04-13 18:08:01,269 INFO namenode.FSNamesystem: Detailed lock hold time metrics enabled: false
2021-04-13 18:08:01,273 INFO namenode.FSNamesystem: fsOwner             = hadoop (auth:SIMPLE)
2021-04-13 18:08:01,273 INFO namenode.FSNamesystem: supergroup          = supergroup
2021-04-13 18:08:01,273 INFO namenode.FSNamesystem: isPermissionEnabled = false
2021-04-13 18:08:01,274 INFO namenode.FSNamesystem: HA Enabled: false
2021-04-13 18:08:01,317 INFO common.Util: dfs.datanode.fileio.profiling.sampling.percentage set to 0. Disabling file IO profiling
2021-04-13 18:08:01,328 INFO blockmanagement.DatanodeManager: dfs.block.

**c**. Creating the HDFS home folder for the `hadoop` user.

In [40]:
!ssh hadoop@{host} 'start-dfs.sh'
!ssh hadoop@{host} 'hadoop fs -mkdir -p /user/hadoop'
!ssh hadoop@{host} 'stop-dfs.sh'

Starting namenodes on [namenode]
Starting datanodes
Starting secondary namenodes [namenode]
Stopping namenodes on [namenode]
Stopping datanodes
Stopping secondary namenodes [namenode]


**d**. Starting HDFS & YARN.

In [54]:
host = NAMENODE_IP
!ssh hadoop@{host} 'start-dfs.sh'
!ssh hadoop@{host} 'start-yarn.sh'

Starting namenodes on [namenode]
Starting datanodes
Starting secondary namenodes [namenode]
Starting resourcemanager
Starting nodemanagers


**e**. Checking all daemons up and running. You should receive 5 lines from the namenode machine and 3 lines from the datanote machines.

In [55]:
for host in VMS:
    run_ssh(host, 'jps')

===== Started on 172.16.3.79 =====
jps
17698 DataNode
18130 Jps
17930 NodeManager
===== Completed on 172.16.3.79 =====
===== Started on 172.16.3.80 =====
jps
13830 Jps
13629 NodeManager
13437 DataNode
===== Completed on 172.16.3.80 =====
===== Started on 172.16.3.81 =====
jps
25249 NodeManager
24050 NameNode
25494 Jps
24537 SecondaryNameNode
24860 ResourceManager
===== Completed on 172.16.3.81 =====


You may check logs at `/opt/hadoop/logs` on the 3 machines and check if everything is alright, or running the `hdfs dfsadmin -report` command (it must return `Live datanodes (3)`).

You can access Hadoop on a browser on your local machine (use IP addresses, not hostnames):
- HDFS subsystem: `http://172.16.3.81:9870/`
- YARN subsystem: `http://172.16.0.1:8088/`

**f.** Run an example provided by Hadoop.
* Wait a minute before running, the daemons should perform some initialization steps
* Ignore initial errors `No such file or directory`
* Ignore logger message by logger `sasl.SaslDataTransferClient`

In [56]:
host = NAMENODE_IP
!ssh hadoop@{host} 'hadoop fs -rm -r input output'
!ssh hadoop@{host} 'hadoop fs -put /opt/hadoop/etc/hadoop/ input'
!ssh hadoop@{host} "hadoop jar /opt/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.1.3.jar grep /user/hadoop/input/*.xml /user/hadoop/output 'dfs[a-z.]+'"
!ssh hadoop@{host} 'hadoop fs -cat output/part-r-00000'

rm: `input': No such file or directory
rm: `output': No such file or directory
2021-04-14 08:35:04,492 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:05,566 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:05,701 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:05,768 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:05,870 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:05,936 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:06,062 INFO sasl.SaslDataTransferClient: SASL encryption trust check: 

2021-04-14 08:35:45,945 INFO client.RMProxy: Connecting to ResourceManager at namenode/172.16.3.81:8032
2021-04-14 08:35:46,033 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1618385672230_0002
2021-04-14 08:35:46,079 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:46,188 INFO input.FileInputFormat: Total input files to process : 1
2021-04-14 08:35:46,241 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:46,364 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:46,408 INFO mapreduce.JobSubmitter: number of splits:1
2021-04-14 08:35:46,468 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-04-14 08:35:46,524 

**g.** Stop HDFS & YARN.

In [57]:
host = NAMENODE_IP
!ssh hadoop@{host} 'stop-dfs.sh'
!ssh hadoop@{host} 'stop-yarn.sh'

Stopping namenodes on [namenode]
Stopping datanodes
Stopping secondary namenodes [namenode]
Stopping nodemanagers
172.16.3.80: WARNING: nodemanager did not stop gracefully after 5 seconds: Trying to kill with kill -9
172.16.3.81: WARNING: nodemanager did not stop gracefully after 5 seconds: Trying to kill with kill -9
172.16.3.79: WARNING: nodemanager did not stop gracefully after 5 seconds: Trying to kill with kill -9
Stopping resourcemanager


## Troubleshooting

If you get an error like the following:
```bash
[2020-01-14 08:48:28.567]Container [pid=155967,containerID=container_1578991625193_0002_01_000023] is running 380426752B beyond the 'VIRTUAL' memory limit. Current usage: 151.6 MB of 1 GB physical memory used; 2.5 GB of 2.1 GB virtual memory used. Killing container.
```
you are using more virtual memory than your current limit of 2.1 Gb. This can be resolved in two ways:

  1. **Disable Virtual Memory Limit Checking**<br>YARN will simply ignore the limit; in order to do this, add this to your `yarn-site.xml` _on each machine_:
      ```bash
      <property>
        <name>yarn.nodemanager.vmem-check-enabled</name>
        <value>false</value>
      </property>
      ```
      The default for this setting is `true`.

  2. **Increase Virtual Memory to Physical Memory Ratio**<br>In your `yarn-site.xml` change this to a higher value than is currently set, _on each machine_:
      ```bash
      <property>
        <name>yarn.nodemanager.vmem-pmem-ratio</name>
        <value>5</value>
      </property>
      ```
      The default is 2.1.<br>
      You could also increase the amount of physical memory you allocate to a container.<br>
      _Make sure you don't forget to restart yarn after you change the configuration_.

# Appendix A. Understanding the Hadoop Memory Allocation <a name="memory"/>

Memory allocation can be complex on low RAM nodes because default values are not suitable for nodes with less than 8 GB of RAM.
In this section we highlight how memory allocation works for MapReduce jobs, and provide a sample configuration for 2 GB RAM nodes.

A YARN job is executed with two kind of resources:

* An **application master** (AM), which is responsible for monitoring the application and coordinating distributed executors in the cluster.
* Some **executors**, that are created by the AM to actually run the job. For a MapReduce job, they will perform map or reduce operation, in parallel.

Both are run in **containers** on **worker nodes**. Each worker node runs a **NodeManager** daemon that is responsible for container creation on the node.
The whole cluster is managed by a **ResourceManager** that schedules container allocation on all the worker nodes, depending on capacity requirements and current charge.

Four types of resource allocations need to be configured properly for the cluster to work. These are:

1. _How much memory can be allocated for YARN containers on a single node._   This limit should be higher than all the others; otherwise, container allocation will be rejected and applications will fail. However, it should not be the entire amount of RAM on the node.
This value is configured in the `yarn-site.xml` file with the `yarn.nodemanager.resource.memory-mb` property.

2. _How much memory a single container can consume and the minimum memory allocation allowed._
A container will never be bigger than the maximum, or else allocation will fail and will always be allocated as a multiple of the minimum amount of RAM.
Those values are configured in the `yarn-site.xml` file with the `yarn.scheduler.maximum-allocation-mb` and `yarn.scheduler.minimum-allocation-mb` properties.

3. _How much memory will be allocated to the ApplicationMaster._
This is a constant value that should fit in the container maximum size.
This value is configured in the `mapred-site.xml` with the `yarn.app.mapreduce.am.resource.mb` property.

4. _How much memory will be allocated to each map or reduce operation._
This should be less than the maximum size.
This value is configured in the `mapred-site.xml` file with the `mapreduce.map.memory.mb` and `mapreduce.reduce.memory.mb` properties.

The relationship between all those properties can be seen in the following figure:
![](https://www.linode.com/docs/databases/hadoop/how-to-install-and-set-up-hadoop-cluster/hadoop-2-memory-allocation-new.png)

For 2 GB nodes, a working configuration may be:

|Property|Value|
|:-------|:----|
|`yarn.nodemanager.resource.memory-mb`  | 1536|
|`yarn.scheduler.maximum-allocation-mb` | 1536|
|`yarn.scheduler.minimum-allocation-mb` |  128|
|`yarn.app.mapreduce.am.resource.mb`    |  512|
|`mapreduce.map.memory.mb`              |  256|
|`mapreduce.reduce.memory.mb`           |  256|

